In [1]:
# If not running on Google Colab/Drive, skip this
# Run this block first and follow the instructions to authorize mounting
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.math import confusion_matrix
from math import ceil
import numpy as np
import pandas as pd
import seaborn as sns
import os
import random
import PIL
import matplotlib.pyplot as plt
import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# If not running on Google Colab/Drive, modify this to root, "."
GDRIVE = os.path.join('/', 'content', 'gdrive', 'My Drive')

In [0]:
RANDOM_SEED = 0

OUTPUT_DATA_DIR = os.path.join(GDRIVE, 'data') # we will store train/ valid/ test/ directories here
TRAIN_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, 'train')
VAL_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, 'valid')
TEST_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, 'test')

WASTE_TYPES = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

IMG_HEIGHT = IMG_WIDTH = 256
EPOCHS = 80
BATCH_SIZE = 32

random.seed(RANDOM_SEED)

In [0]:
# So that we can visualize some of the augmented images
TRAIN_AUG_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, 'train_aug')     

if not os.path.exists(TRAIN_AUG_DATA_DIR):
    os.makedirs(TRAIN_AUG_DATA_DIR)

In [0]:
MODEL_NAME = 'small_cnn_augmented'

def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH , 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(len(WASTE_TYPES), activation='softmax')
    ])
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [24]:
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 125, 125, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 30, 30, 64)       

In [25]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=90,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# using default params: target_size=(256, 256)
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR, batch_size=BATCH_SIZE, save_to_dir=TRAIN_AUG_DATA_DIR)
val_generator = val_datagen.flow_from_directory(VAL_DATA_DIR, batch_size=BATCH_SIZE)
# don't shuffle test set, it will be easier to predict then
test_generator = test_datagen.flow_from_directory(TEST_DATA_DIR, batch_size=BATCH_SIZE, shuffle=False)

# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

#sample_training_images, _ = next(train_generator)
#plotImages(sample_training_images[:5])

Found 1514 images belonging to 6 classes.
Found 505 images belonging to 6 classes.
Found 508 images belonging to 6 classes.


In [26]:
# Get these values from the above cell, or from prepare_train_test_data.ipynb
total_train = 1514
total_val = 505
total_test = 508
print(total_train, total_val, total_test)

1514 505 508


In [27]:
history = model.fit(
    train_generator,
    steps_per_epoch=ceil(total_train / BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=ceil(total_val / BATCH_SIZE),
    callbacks=[ModelCheckpoint(os.path.join(GDRIVE, f'{MODEL_NAME}.h5'), save_best_only=True, save_weights_only=True, verbose=1)])

Epoch 1/80
48/48 [==============================] - ETA: 0s - loss: 2.0502 - accuracy: 0.1982 
Epoch 00001: val_loss improved from inf to 1.73930, saving model to /content/gdrive/My Drive/small_cnn_augmented.h5
48/48 [==============================] - 694s 14s/step - loss: 2.0502 - accuracy: 0.1982 - val_loss: 1.7393 - val_accuracy: 0.2356
Epoch 2/80
48/48 [==============================] - ETA: 0s - loss: 1.7439 - accuracy: 0.2285
Epoch 00002: val_loss did not improve from 1.73930
48/48 [==============================] - 71s 1s/step - loss: 1.7439 - accuracy: 0.2285 - val_loss: 1.7700 - val_accuracy: 0.2099
Epoch 3/80
48/48 [==============================] - ETA: 0s - loss: 1.7310 - accuracy: 0.2120
Epoch 00003: val_loss improved from 1.73930 to 1.66976, saving model to /content/gdrive/My Drive/small_cnn_augmented.h5
48/48 [==============================] - 71s 1s/step - loss: 1.7310 - accuracy: 0.2120 - val_loss: 1.6698 - val_accuracy: 0.2495
Epoch 4/80
48/48 [=======================

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs_range = range(EPOCHS)

plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Learning Curve, small CNN with augmented data')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.show()
plt.savefig(os.path.join(GDRIVE, f'{MODEL_NAME}_learning_curve.png'))
plt.close()

with open(os.path.join(GDRIVE, f'{MODEL_NAME}_history.pickle'), 'wb') as f:
     pickle.dump(history.history, f)

In [ ]:
model = create_model() # create a new instance
model.load_weights(os.path.join(GDRIVE, f'{MODEL_NAME}.h5'))

In [30]:
test_predictions = np.argmax(model.predict(test_generator, verbose=1, batch_size=BATCH_SIZE, steps=ceil(total_test / BATCH_SIZE)), axis=-1)

16/16 [==============================] - 146s 9s/step


In [0]:
test_cm = np.array(confusion_matrix(test_generator.labels, test_predictions))
df_cm = pd.DataFrame(test_cm, WASTE_TYPES, WASTE_TYPES)

plt.figure(figsize=(10, 8))
sns.heatmap(df_cm, annot=True, fmt="d", cmap="YlGnBu")
plt.savefig(os.path.join(GDRIVE, f'{MODEL_NAME}_confusion_matrix.png'))
plt.close()

In [32]:
accuracy = np.trace(test_cm) / np.sum(test_cm)
print(f'Test accruracy: {accuracy * 100:.2f}%')

Test accruracy: 62.60%
